In [1]:
import pandas as pd
import json
import re
from collections import Counter
import presidio_analyzer
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern
# Define the file path
file_path = r"C:\Users\draph\Downloads\train.json\train.json"

# Load the JSON file into a Python dictionary
with open(file_path, 'r') as f:
    data = json.load(f)
# Assuming 'data' is the loaded JSON data
# Convert the JSON data to a DataFrame
df = pd.DataFrame(data)
print(df[0:5])

C:\Users\draph\AppData\Local\Temp\ipykernel_120408\775007738.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


   document                                          full_text  \
0         7  Design Thinking for innovation reflexion-Avril...   
1        10  Diego Estrada\n\nDesign Thinking Assignment\n\...   
2        16  Reporting process\n\nby Gilberto Gamboa\n\nCha...   
3        20  Design Thinking for Innovation\n\nSindy Samaca...   
4        56  Assignment:  Visualization Reflection  Submitt...   

                                              tokens  \
0  [Design, Thinking, for, innovation, reflexion,...   
1  [Diego, Estrada, \n\n, Design, Thinking, Assig...   
2  [Reporting, process, \n\n, by, Gilberto, Gambo...   
3  [Design, Thinking, for, Innovation, \n\n, Sind...   
4  [Assignment, :,   , Visualization,  , Reflecti...   

                                 trailing_whitespace  \
0  [True, True, True, True, False, False, True, F...   
1  [True, False, False, True, True, False, False,...   
2  [True, False, False, True, True, False, False,...   
3  [True, True, True, False, False, True, 

In [3]:
#print(df['tokens','trailing_whitespace','labels'].apply(lambda x: x[0] if len(x) > 0 else None))
df2 = pd.DataFrame()
df2=df.loc[:, ['tokens', 'trailing_whitespace', 'labels']].apply(lambda x: x[0] if len(x) > 0 else None)
print(df2[0:13])

#df2[1]=df.loc[:, ['tokens', 'trailing_whitespace', 'labels']].apply(lambda x: x[1] if len(x) > 0 else None)
#df2[2]=df.loc[:, ['tokens', 'trailing_whitespace', 'labels']].apply(lambda x: x[2] if len(x) > 0 else None)

        tokens  trailing_whitespace          labels
0       Design                 True               O
1     Thinking                 True               O
2          for                 True               O
3   innovation                 True               O
4    reflexion                False               O
5            -                False               O
6        Avril                 True               O
7         2021                False               O
8            -                False               O
9     Nathalie                 True  B-NAME_STUDENT
10       Sylla                False  I-NAME_STUDENT
11        \n\n                False               O
12   Challenge                 True               O


In [4]:
# Create an empty DataFrame to store the results
df2 = pd.DataFrame()

# Loop through indices (0, 1, 2) to extract elements from each row
for i in range(3):  # Assuming there are 3 elements in each row (tokens, trailing_whitespace, labels)
    # Extract the ith element from each column for all rows
    df2[i] = df.loc[:, ['tokens', 'trailing_whitespace', 'labels']].apply(lambda x: x[i] if len(x) > i else None)

# Print the resulting DataFrame
print(df2)

ValueError: Cannot set a DataFrame with multiple columns to the single column 0

## Print an Essay

In [5]:
print(df['full_text'][0])

Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla

Challenge & selection

The tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.

What exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1

This tool has many advantages:

•  It is accessible to all and does not require significant material investment and can be done  quickly

•  It is scalable

•  It allows categorization and linking of information

•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas

•  It is suitable for all people and is easy to learn

•  It is fun and encourages exchanges

•  It makes visible the dimension of pr

## Analyze Essay->Presidio

In [17]:
analyzer=AnalyzerEngine()
analyzer_results=analyzer.analyze(df['full_text'][0],language="en")

def Type_grab(string1):
    # Split the string by comma and whitespace
    result_parts = string1.split(", ")

    # Iterate through the parts and find the one that starts with "type: "
    entity_type = None
    for part in result_parts:
        if part.startswith("type: "):
            # Extract the type by removing "type: " prefix
            entity_type = part.replace("type: ", "")
            break
    print("Entity type:", entity_type)

presidio_output = [
    {"text": "John Doe", "start": 0, "end": 8, "entity_type": "PERSON"},
    {"text": "New York", "start": 10, "end": 18, "entity_type": "LOCATION"},
    {"text": "2024-01-01", "start": 20, "end": 29, "entity_type": "DATE"},
    {"text": "john.doe@example.com", "start": 31, "end": 52, "entity_type": "EMAIL_ADDRESS"}
]

# Extract entities with their types
parsed_entities = [(entity["text"], entity["entity_type"]) for entity in presidio_output]

# Display the extracted entities and their types
for entity, entity_type in parsed_entities:
    print(f"Entity: {entity}, Type: {entity_type}")
analyzer_results[0].getProperties()   
#print(Type_grab(analyzer_results[0].Getstring()))    
#pd.DataFrame(analyzer_results[0])
for res in analyzer_results:
    print(res)
    

AttributeError: 'RecognizerResult' object has no attribute 'getProperties'

## Book citation

In [42]:
# Define specific words to search for
specific_words = ['Design Thinking', 'economics','2020', '2021', 'source','Sources', 'Reference', 'References', 'Challenge', 'Approach']

# Function to count occurrences of specific words in all essays
def count_specific_words_in_all_essays(df, words):
    word_counts = Counter()
    for essay_text in df['full_text']:
        for word in words:
            word_counts[word] += essay_text.lower().count(word.lower())
    return dict(word_counts)

# Function to count the number of essays containing each specific word
def count_essays_containing_specific_words(df, words):
    essay_counts = {word: sum(word.lower() in essay.lower() for essay in df['full_text']) for word in words}
    return essay_counts

# Get the count of essays containing each specific word
essay_counts = count_essays_containing_specific_words(df, specific_words)

# Get the total word counts across all essays
total_word_counts = count_specific_words_in_all_essays(df, specific_words)
# Print the total word counts
print(total_word_counts)

# Print the counts
print(essay_counts)
len(df)

{'Design Thinking': 6283, 'economics': 58, '2020': 179, '2021': 99, 'source': 2277, 'Sources': 1550, 'Reference': 980, 'References': 450, 'Challenge': 15113, 'Approach': 11668}
{'Design Thinking': 3164, 'economics': 43, '2020': 129, '2021': 72, 'source': 1447, 'Sources': 1088, 'Reference': 799, 'References': 379, 'Challenge': 6357, 'Approach': 6205}


6807